In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.applications import VGG16

In [2]:
# Load the fashion_mnist dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [4]:
print(f'x_train shape: {x_train.shape}')
print(f'y_train shape: {y_train.shape}')

x_train shape: (60000, 28, 28)
y_train shape: (60000,)


In [3]:
# Normalize pixel values to be between 0 and 1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [5]:
# Add a channel dimension to the images
x_train = tf.expand_dims(x_train, -1)
x_test = tf.expand_dims(x_test, -1)

In [7]:
print(f'x_train shape: {x_train.shape}')
print(f'y_test shape: {x_test.shape}')

x_train shape: (60000, 28, 28, 1)
y_test shape: (10000, 28, 28, 1)


In [24]:
#prepare x_train and x_test for the VGG16 model (100, 100, 3) 4D tensor (batch, height, width, channels)
import numpy as np

x_train_3 = np.repeat(x_train, 3, -1)
x_test_3 = np.repeat(x_test, 3, -1)
x_train_3 = tf.image.resize(x_train_3, (100, 100))
x_test_3 = tf.image.resize(x_test_3, (100, 100))
print(f'x_train_3 shape: {x_train_3.shape}')
print(f'x_test_3 shape: {x_test_3.shape}')

x_train_3 shape: (60000, 100, 100, 3)
x_test_3 shape: (10000, 100, 100, 3)


In [25]:
# Define the VGG16 model
vgg16 = VGG16(
    include_top=False,
    weights=None,
    input_shape=(100, 100, 3),
    pooling='max'
)

In [26]:
# Freeze the weights of the VGG16 model
for layer in vgg16.layers:
    layer.trainable = False

In [27]:
# Add a classification head to the VGG16 model
inputs = keras.Input(shape=(100, 100, 3))
x = vgg16(inputs, training=False)
x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation="softmax")(x)

In [28]:
# Compile the model
model = keras.Model(inputs, outputs)

In [29]:
# Print the model summary
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                2570      
                                                                 
Total params: 14848586 (56.64 MB)
Trainable params: 133898 

In [30]:
# Compile the model
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

In [ ]:
# Train the model
model.fit(x_train_3, y_train, batch_size=128, epochs=10, validation_split=0.1)

Epoch 1/10
 70/422 [===>..........................] - ETA: 1:17:24 - loss: 2.3003 - accuracy: 0.1088

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test_3, y_test)
print("Test accuracy:", test_acc)

In [ ]:
# Save the model
model.save("model2.h5")

In [ ]:
# Load the model
model = keras.models.load_model("model2.h5")


In [ ]:
# Make predictions on a sample of test images
predictions = model.predict(x_test_3[:10])

In [ ]:
# Print model predictions
print(f"Predictions:\n {predictions}\n")
print(f"Predictions shape: {predictions.shape}")

In [ ]:
# Print the true labels
print(f"True labels:\n {y_test[:10]}")

In [ ]:
# Plot the first 10 test images and their predicted labels
import matplotlib.pyplot as plt
#plot the first 10 test images, their predicted labels, and the true labels
#color correct predictions in green, incorrect predictions in red
for i in range(10):
    plt.grid(False)
    plt.imshow(x_test_3[i, :, :, 0], cmap=plt.cm.binary)
    plt.xlabel(f"Actual: {y_test[i]}")
    plt.title(f"Prediction: {np.argmax(predictions[i])}")
    plt.show()